In [ ]:
#Paquetes necesarios
pip install -r requirements.txt

In [1]:
#Paquetes necesarias para el preprocesamiento
import mne
from mne_bids import BIDSPath, read_raw_bids
import matplotlib.pyplot as plt
import pandas as pd 
import mne_connectivity
import seaborn as sns
import numpy as np

In [2]:
#Importar función de los autores
from technical_validation_utils import (get_raw_condition, plot_individual_coh_topo, plot_coh_topo)
from config import bids_root

In [3]:
session = 'PeriOp'

datatype = 'meg'

subjects = ['0cGdk9', '2IU8mi', 'AB2PeX', 'AbzsOg', 
            'dCsWjQ', 'FYbcap', 'gNX5yb', 'hnetKS', 'i4oK0F',
            'jyC0j3', '-oLNpHd', 'PuPVlx', 'QZTsn6']

acquisitions = ['MedOff', 'MedOn']

tasks = ['MoveL', 'HoldL']

run = '1'

dict_matrix = {}
dict_df = {}

for subject in subjects:
    for acquisition in acquisitions:
        for task in tasks:

            bids_path = BIDSPath(root=bids_root, session=session, subject=subject, datatype=datatype, task=task,
                     acquisition=acquisition, run=run, extension='.fif')

            #Leer los datos crudos
            raw = read_raw_bids(bids_path)

            # Escoger los tipos de canales deseados. EEG = LFP
            channel_to_pick = {'meg':'grad', 'eeg': True, 'emg': False, 'eog': False, 'stim': False}
            
            # Escoger todos los canales y excluir los malos
            raw.pick_types(**channel_to_pick, exclude='bads')
            
            # create a bids path for the montage, check=False because this is not a standard bids file
            montage_path = BIDSPath(root=bids_root, session=session, subject=subject,
                                    datatype='montage', extension='.tsv', check=False)
            
            # get the montage tsv bids file 
            montage = montage_path.match()[0]
            # read the file using pandas
            df = pd.read_csv(montage, sep='\t')
            
            # create a dictionary mapping old names to new names for right and left channels
            montage_mapping = {row['right_contacts_old']: row['right_contacts_new'] for idx, row in df.iterrows()} # right
            montage_mapping.update({row['left_contacts_old']: row['left_contacts_new'] for idx, row in df.iterrows()}) # left
            
            # remove in the montage mapping the channels that are not in the raw anymore (because they were marked as bads)
            montage_mapping  = {key: value for key, value in montage_mapping.items() if key in raw.ch_names }
            print(montage_mapping)
            
            # rename the channels using the montage mapping scheme
            raw.rename_channels(montage_mapping)
            
            # get list of lfp names and their side for bipolar contact
            lfp_name = [name for name in raw.ch_names if 'LFP' in name]
            lfp_right = [name for name in lfp_name if 'right' in name]
            lfp_left = [name for name in lfp_name if 'left' in name]
            
            # get LFP-side-contact then add the next contact number to the name to create the new bipolar name
            # e.g. --> LFP-right-0 and LFP-right-1 will become LFP-right-01
            if len(lfp_right) > 1: # we need at least 2 contact to create a bipolar scheme
                bipolar_right = [f'{lfp_right[i]}{lfp_right[i+1][-1]}' for i in range(len(lfp_right)-1)]
                ref_right = True
            else:
                ref_right = False # else do no re-reference this side
            
            if len(lfp_left) > 1:
                bipolar_left = [f'{lfp_left[i]}{lfp_left[i+1][-1]}' for i in range(len(lfp_left)-1)]
                ref_left = True
            else: 
                ref_left = False
            
            # Cargar datos
            raw.load_data()
            
            # set bipolar reference scheme for each respective side 
            if ref_right:
                raw = mne.set_bipolar_reference(raw, anode=lfp_right[:-1], cathode=lfp_right[1:], ch_name=bipolar_right)
                
            if ref_left:
                raw = mne.set_bipolar_reference(raw, anode=lfp_left[:-1], cathode=lfp_left[1:], ch_name=bipolar_left)
            
            # apply a 1Hz high pass firwin filter to remove slow drifts
            raw.filter(l_freq=1, h_freq=None, method='fir', n_jobs=-1)
            
            # after filtering, now do downsample to 200 from 2000. NB: downsample after filter so no aliasing introduced
            raw.resample(200)
            
            # prepare the conditions to crop the data and get the rest_segments and task_segments
            conditions = ['rest'] + [bids_path.task] # rest + task (hold or move)
            rest_segments, task_segments = get_raw_condition(raw, conditions)
            segments = [("rest", rest_segments), ("task", task_segments)]

            for seg_name, segment in segments:
                # from now on we will focus only in the resting state segment we just cropped
                # create epochs of fixed length 2 seconds and 50% overlap.
                epochs = mne.make_fixed_length_epochs(segment, duration=2, overlap=1, preload=True)
            
                # get the sampling freq
                sfreq = epochs.info['sfreq']
                
                # create epochs with only meg or lfp
                meg = epochs.copy().pick_types(meg='grad')
                lfp = epochs.copy().pick_types(eeg=True)
                
                # get the LFP channel names for right and left side with the bipolar scheme
                lfp_right = [name for name in lfp.ch_names if 'right' in name]
                lfp_left = [name for name in lfp.ch_names if 'left' in name]
                    
                # find the indices of the desired channel names for averaging later
                lfp_right_ind = [lfp.ch_names.index(name) for name in lfp_right]
                lfp_left_ind = [lfp.ch_names.index(name) for name in lfp_left]
                
                # get the epochs data to compute connectivity
                data = epochs.get_data(copy=True)

                #Data está en la forma (n_epochs,n_nodos,n_puntos por epoch (sampling freq * duración epoch)), se reordena el array para que cada fila
                #sea cada nodo y se colapsa a dos dimensiones para que en lugar estar cortando por cada epoch se obtenga una serie continua de todos
                #los puntos en el tiempo de cada canal. Esto tendría la forma n_nodos, n_epochs*n_puntos por epoch. Así ya es posible graficar
                #y correlacionar.

                n_epochs, n_channels, n_times = data.shape

                # Concatenamos todos los epochs en el eje del tiempo
                data_concat = data.transpose(1, 0, 2).reshape(n_channels, n_epochs * n_times)
                corr_matrix = np.corrcoef(data_concat)
                key_matrix = f"corr_matrix_{subject}_{acquisition}_{task}_{seg_name}"
                dict_matrix[key_matrix] = corr_matrix
                sz = range(corr_matrix.shape[0])
                corr_df = pd.DataFrame(data = corr_matrix, index = sz, columns = sz)
                key_df = f"corr_df_{subject}_{acquisition}_{task}_{seg_name}"
                dict_df[key_df] = corr_df

                corr_df.to_csv(f"{key_df}.csv")

Opening raw data file C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-0cGdk9\ses-PeriOp\meg\sub-0cGdk9_ses-PeriOp_task-MoveL_acq-MedOff_run-1_meg.fif...
    Read a total of 8 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
    Range : 114000 ... 1355999 =     57.000 ...   678.000 secs
Ready.
Reading events from C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-0cGdk9\ses-PeriOp\meg\sub-0cGdk9_ses-PeriOp_task-MoveL_acq-MedOff_run-1_events.tsv.
Reading channel info from C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-0cGdk9\ses-PeriOp\meg\

C:\Users\PC\AppData\Local\Temp\ipykernel_12104\1536407131.py:26: RuntimeWarning: Unable to map the following column(s) to to MNE:
hand (subject handedness): unknown
disease: ['Parkinson Disease']
diseaseDuration: [10]
  raw = read_raw_bids(bids_path)


EEG channel type selected for re-referencing
Created an SSP operator (subspace dimension = 2)
Creating RawArray with float64 data, n_channels=2, n_times=1242000
    Range : 114000 ... 1355999 =     57.000 ...   678.000 secs
Ready.


C:\Users\PC\AppData\Local\Temp\ipykernel_12104\1536407131.py:78: RuntimeWarning: Projection vector 'grad_ssp_upright.fif : PCA-v1' has been reduced to 0.01% of its original magnitude by subselecting 202/204 of the original channels. If the ignored channels were bad during SSP computation, we recommend recomputing proj (via compute_proj_raw or related functions) with the bad channels properly marked, because computing SSP with bad channels present in the data but unmarked is dangerous (it can bias the PCA used by SSP). On the other hand, if you know that all channels were good during SSP computation, you can safely use info.normalize_proj() to suppress this warning during projection.
  raw = mne.set_bipolar_reference(raw, anode=lfp_right[:-1], cathode=lfp_right[1:], ch_name=bipolar_right)


Added the following bipolar channels:
LFP-right-12, LFP-right-23
EEG channel type selected for re-referencing
Created an SSP operator (subspace dimension = 2)
Creating RawArray with float64 data, n_channels=3, n_times=1242000
    Range : 114000 ... 1355999 =     57.000 ...   678.000 secs
Ready.


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Not setting metadata
338 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
8 projection items activated
Using data from preloaded Raw for 338 events and 400 original time points ...
8 bad epochs dropped


C:\Users\PC\AppData\Local\Temp\ipykernel_12104\1536407131.py:97: RuntimeWarning: Projection vector 'grad_ssp_upright.fif : PCA-v1' has been reduced to 0.01% of its original magnitude by subselecting 202/204 of the original channels. If the ignored channels were bad during SSP computation, we recommend recomputing proj (via compute_proj_raw or related functions) with the bad channels properly marked, because computing SSP with bad channels present in the data but unmarked is dangerous (it can bias the PCA used by SSP). On the other hand, if you know that all channels were good during SSP computation, you can safely use info.normalize_proj() to suppress this warning during projection.
  epochs = mne.make_fixed_length_epochs(segment, duration=2, overlap=1, preload=True)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Opening raw data file C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOn_run-1_meg.fif...
    Read a total of 8 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
    Range : 106500 ... 1760999 =     53.250 ...   880.500 secs
Ready.
Opening raw data file C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_ta

C:\Users\PC\AppData\Local\Temp\ipykernel_12104\1536407131.py:26: RuntimeWarning: Expected to find a single events.tsv file associated with sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOn_run-1, but found 6:

C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOff_run-1_events.tsv
C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOn_run-1_split-01_events.tsv
C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOn_run-1_split-02_events.tsv
C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOn_run-2_events.tsv
C:\Users\PC\Desktop\Neurociencias\Modelos Computacionales I\Data\sub-2IU8mi\ses-PeriOp\meg\sub-2IU8mi_ses-PeriOp_task-MoveL_acq-MedOff_run-1_events.tsv
C:\Users\PC\Desktop\Ne

ValueError: list.index(x): x not in list. Did you mean one of ['meg\\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOn_run-2_meg.fif', 'meg\\sub-2IU8mi_ses-PeriOp_task-HoldL_acq-MedOff_run-1_meg.fif', 'meg\\sub-2IU8mi_ses-PeriOp_task-MoveL_acq-MedOn_run-1_meg.fif']?

In [ ]:
#Matriz de n_canales x n_canales x frecuencia
print(len(coh_lfpmeg.freqs),"Frecuencias (Hz):", coh_lfpmeg.freqs)
mat = coh_lfpmeg.get_data(output='dense')
print('Matriz inicial:',mat.shape)

#Promediar la matriz sobre la tercera dimensión (0,1,2)
mat = coh_lfpmeg.get_data(output='dense').mean(axis=2)
#Eliminar diagonal
mat = mat - np.eye(mat.shape[0])
#hmap = sns.heatmap(mat, annot = False, cmap = 'hot')
mat_df = pd.DataFrame(data = mat, index = all_idx, columns = all_idx)
mat_df
#print(mat_df)

In [ ]:
hmap = sns.heatmap(mat_df, annot = False)

In [ ]:
a = np.mean(matrix.values) + np.std(matrix.values)

In [ ]:
#Convertir valores NaN a 0
matrix = matrix[matrix > a[0]]

i = 0
for value in matrix.values.ravel():
    if np.isnan(value) == True:
        matrix.values.ravel()[i] = 0
    i += 1

In [ ]:
montage